In [1]:
import os
import sys
import pandas as pd
import numpy as np
import time
import tables
import warnings
from datetime import datetime, timedelta
from pvoutput import *

In [2]:
INPUT_PV_LIST_FILENAME = '/home/jack/data/pvoutput.org/processed/UK_PV_listing_metadata.hdf'
METADATA_FILENAME = '/home/jack/data/pvoutput.org/processed/UK_PV_metadata.csv'
OUTPUT_TIMESERIES_FILENAME = '/home/jack/data/pvoutput.org/processed/UK_PV_timeseries.hdf'
PV_STATS_FILENAME = '/home/jack/data/pvoutput.org/processed/UK_PV_stats.csv'
MISSING_DATA_FILENAME = '/home/jack/data/pvoutput.org/processed/UK_PV_missing_timeseries_data.csv'
START_DATE = pd.Timestamp("2019-01-01")
END_DATE = pd.Timestamp("2019-07-01")

logger = get_logger(stream_handler=False)

In [3]:
pv_systems = pd.read_hdf(INPUT_PV_LIST_FILENAME, 'metadata')

In [4]:
pv_systems.head()

,Array Tilt,Inverter,Location,Orientation,Panels,Shade,system_capacity,system_name,Outputs,Generation,Average,Efficiency,system_capacity_kw,efficiency_kWh_per_kW,system_1_tilt_degrees,system_2_tilt_degrees
system_id,,,,,,,,,,,,,,,,
26965,11.0°,Enphase M215,United Kingdom CV47,South 0.0°,336x250W Q Cells Pro-G3 250,Low,84.000kW,mfl_phc,1813 days,399.238MWh,220.208kWh,2.654kWh/kW,84.00,2.654,11.0,NaN
24768,NaN,aurora trio,United Kingdom OX7,South 180.0°,1050x240W qidong,No,252.000kW,mfl_qfa,446 days,307.029MWh,688.405kWh,2.732kWh/kW,252.00,2.732,NaN,NaN
11542,15.0°,SMA TRI-Power 1700 TL,United Kingdom PE11,South 180.0°,200x250W Emmvee ES 230 M60 B,No,50.000kW,Wray Farms Solar System,2437 days,293.684MWh,120.510kWh,2.477kWh/kW,50.00,2.477,15.0,NaN
66991,40.0°,Unknown,United Kingdom HR8,South 180.0° / South West 225.0°,152x325W + 80x325W Unknown,No,75.400kW,Ledbury Community Hospital,1434 days,279.902MWh,195.190kWh,2.603kWh/kW,75.40,2.603,40.0,NaN
5116,15.0°,Fronius CL36; Datamanager 2.0,United Kingdom OX7,South 180.0°,182x235W Q.Base-G2 235,No,42.770kW,mfl_scf,2538 days,267.470MWh,105.386kWh,2.493kWh/kW,42.77,2.493,15.0,NaN


In [ ]:
pv_metadata = pd.read_csv(METADATA_FILENAME, index_col='system_id')

In [ ]:
pv_metadata.head()

,system_name,system_size_watts,postcode,number_of_panels,panel_power_watts,panel_brand,num_inverters,inverter_power_watts,inverter_brand,orientation,array_tilt_degrees,shade,install_date,latitude,longitude,status_interval_minutes,number_of_panels_secondary,panel_power_watts_secondary,orientation_secondary,array_tilt_degrees_secondary
system_id,,,,,,,,,,,,,,,,,,,,
19397,AndyT's,3900,BS22,20,195,ZN Shine 195w,1,3500,Kaco 4202,S,30.0,No,2011-11-21 00:00:00,51.36,-2.92,5,0,0,NaN,NaN
8195,Kilmarnock Roof,3750,KA3,15,250,Sanyo 250 HIT,1,4000,Omniksol,S,30.0,No,2011-11-07 00:00:00,55.64,-4.49,10,0,0,NaN,NaN
8200,Flat 5,3430,E8,14,245,NaN,1,3000,sb3000,S,25.0,Low,2011-12-12 00:00:00,51.54,-0.06,5,0,0,NaN,NaN
8204,Sooper-Dooper Solar,2940,GU2,12,245,SunTech STP245S-20/Wd,1,3000,Kaco Powador 3002,S,19.0,No,2012-05-11 00:00:00,51.24,-0.59,10,0,0,NaN,NaN
8205,58GPR,4000,BS48,16,250,Sanyo component Europe GmbH,1,50,Sma sunny boysb3800v,S,NaN,No,2011-11-10 00:00:00,51.42,-2.74,10,0,0,NaN,NaN


In [ ]:
pv_stats = pd.read_csv(
    PV_STATS_FILENAME, 
    index_col='system_id',
    parse_dates=['actual_date_from', 'actual_date_to', 'record_efficiency_date']
)
pv_stats.head()

,energy_generated_Wh,energy_exported_Wh,average_generation_Wh,minimum_generation_Wh,maximum_generation_Wh,average_efficiency_kWh_per_kW,outputs,actual_date_from,actual_date_to,record_efficiency_kWh_per_kW,record_efficiency_date,stats_downloaded_on_utc
system_id,,,,,,,,,,,,
12290,28040596.0,0.0,10068.0,0.0,28548.0,2.543,2785.0,2011-11-30,2019-07-21,570.96,2013-05-25 00:00:00,1.563877e+18
8200,6399918.0,77311.0,7950.0,2.0,31047.0,2.388,805.0,2012-05-01,2014-10-14,620.94,2012-09-08 00:00:00,1.563877e+18
4105,12214111.0,7482325.0,9239.0,0.0,44085.0,2.924,1322.0,2011-10-11,2015-05-24,881.70,2015-03-23 00:00:00,1.563877e+18
53258,0.0,0.0,0.0,0.0,0.0,0.000,584.0,2017-07-15,2019-02-20,0.00,2019-02-20 00:00:00,1.563877e+18
8205,4313500.0,0.0,10546.0,0.0,27500.0,210.920,409.0,2011-11-11,2012-12-23,550.00,2012-05-16 00:00:00,1.563877e+18


In [ ]:
pv_systems_joined = (
    pv_systems
    .join(
        pv_metadata[['status_interval_minutes', 'install_date', 'latitude', 'longitude']],
        how='left'
    ).join(
        pv_stats[['actual_date_from', 'actual_date_to', 'outputs']],
        how='left'
    ))

In [ ]:
# Filter 'bad' systems
pv_systems_filtered = pv_systems_joined.query(
    'status_interval_minutes <= 60 and'
    ' outputs > 100 and'
    ' actual_date_from <= @START_DATE and'
    ' actual_date_to >= @END_DATE')
pv_systems_filtered = pv_systems_filtered.dropna(subset=['latitude', 'longitude'])

In [ ]:
len(pv_systems_filtered)

829

In [ ]:
pv_systems_filtered.head()

,Array Tilt,Inverter,Location,Orientation,Panels,Shade,system_capacity,system_name,Outputs,Generation,...,efficiency_kWh_per_kW,system_1_tilt_degrees,system_2_tilt_degrees,status_interval_minutes,install_date,latitude,longitude,actual_date_from,actual_date_to,outputs
system_id,,,,,,,,,,,,,,,,,,,,,
26965,11.0°,Enphase M215,United Kingdom CV47,South 0.0°,336x250W Q Cells Pro-G3 250,Low,84.000kW,mfl_phc,1813 days,399.238MWh,...,2.654,11.0,NaN,5,2014-01-17 00:00:00,52.230000,-1.380000,2014-01-08,2019-07-23,1815.0
11542,15.0°,SMA TRI-Power 1700 TL,United Kingdom PE11,South 180.0°,200x250W Emmvee ES 230 M60 B,No,50.000kW,Wray Farms Solar System,2437 days,293.684MWh,...,2.477,15.0,NaN,5,2011-12-08 00:00:00,52.800000,-0.160000,2012-09-12,2019-07-23,2439.0
5116,15.0°,Fronius CL36; Datamanager 2.0,United Kingdom OX7,South 180.0°,182x235W Q.Base-G2 235,No,42.770kW,mfl_scf,2538 days,267.470MWh,...,2.493,15.0,NaN,10,2011-12-07 00:00:00,51.945568,-1.385417,2011-12-09,2019-07-23,2540.0
5701,1.0°,SMA Tripower 8000TL,United Kingdom DA1,South 180.0°,46x215W Schuco,Low,9.890kW,Workshop,2837 days,78.561MWh,...,2.868,1.0,NaN,5,NaN,51.420000,0.110000,2011-10-15,2019-07-23,2839.0
28016,30.0°,SMA Sunny Boy 17000TL,United Kingdom CB3,South 180.0° / South East 135.0°,80x240W + 44x240W REW 240 Premium QM60,No,29.760kW,Robinson College,772 days,77.214MWh,...,3.383,30.0,NaN,5,2013-10-10 00:00:00,52.210000,0.020000,2014-03-22,2019-07-23,774.0


In [ ]:
logger.info('\n******* STARTING UP ************')

# SETUP: Find which dates, and for which system ids we already have data:
if os.path.exists(OUTPUT_TIMESERIES_FILENAME):
    date_ranges_already_downloaded = {}  # {system_id: (start_date, end_date)}
    with pd.HDFStore(OUTPUT_TIMESERIES_FILENAME, mode='r') as store:
        keys = store.keys()
        for key in keys:
            if key == '/missing_dates':
                continue
            datetimes = store.select(key=key, columns=['datetime'])
            index = datetimes.sort_index().index
            system_id = int(key.replace('/', ''))
            date_ranges_already_downloaded[system_id] = (index[0].date(), index[-1].date())
    already_processed = date_ranges_already_downloaded.keys()
else:
    already_processed = []
    
print(len(already_processed), 'already downloaded')
print(pd.DataFrame(date_ranges_already_downloaded).T)


# Get list of missing data
with pd.HDFStore(OUTPUT_TIMESERIES_FILENAME, mode='r') as store:
    try:
        all_missing_dates = store['missing_dates']
    except KeyError:
        all_missing_dates = pd.DataFrame()

print('Number of missing dates found so far:', len(all_missing_dates))


system_id = 0  # Just so logger.exception has something to process if necessary.
try:
    n = len(pv_systems_filtered)
    api_request_i = 0
    for i, system_id in enumerate(pv_systems_filtered.index):
        msg = 'system_id {} of {}: {}  '.format(i, n, system_id)
        print('\r', msg, end='', flush=True)
        logger.info(msg)
        date_range = pd.date_range(START_DATE, END_DATE, freq="D", normalize=True)
        if system_id in date_ranges_already_downloaded:
            date_range_downloaded = date_ranges_already_downloaded[system_id]
            date_range_downloaded = pd.date_range(
                date_range_downloaded[0], date_range_downloaded[1], freq="D", normalize=True)
            date_range = set(date_range) - set(date_range_downloaded)

        # Check for dates that we've already requested, but are missing:
        try:
            missing_dates_for_id = all_missing_dates.loc[system_id]
        except KeyError:
            pass
        else:
            missing_dates_for_id = missing_dates_for_id['date']
            if isinstance(missing_dates_for_id, pd.Timestamp):
                missing_dates_for_id = [missing_dates_for_id]
            else:
                missing_dates_for_id = missing_dates_for_id.to_list()
            logger.info('    Missing dates already found for %d: %s', system_id, missing_dates_for_id)
            date_range = set(date_range) - set(missing_dates_for_id)

        date_range = np.sort(list(date_range))
        logger.info("  %d days to download", len(date_range))
        if len(date_range) > 0:
            logger.info("    from %s to %s", date_range[0], date_range[-1])
        for date in date_range:
            api_request_i += 1
            logger.info("  system_id: %d; Date: %s;  API request number: %d", system_id, date, api_request_i)
            try:
                timeseries = get_pv_system_status(system_id, date)
            except NoStatusFound as e:
                logger.info("  system_id: %d: No status found for %s.", system_id, date)
                missing_dates = pd.DataFrame({'date': date}, index=[system_id])
                with pd.HDFStore(OUTPUT_TIMESERIES_FILENAME, mode='a', complevel=9) as store:
                    store.append(key='missing_dates', value=missing_dates)
            else:
                logger.info("  system_id: %d; %d rows retrieved", system_id, len(timeseries))
                with pd.HDFStore(OUTPUT_TIMESERIES_FILENAME, mode='a', complevel=9) as store:
                    with warnings.catch_warnings():
                        warnings.simplefilter('ignore', tables.NaturalNameWarning) 
                        store.append(key=str(system_id), value=timeseries)
except:
    logger.exception('Exception whilst processing system_id %d', system_id)

85 already downloaded
                0           1
10156  2019-06-01  2019-07-01
11542  2019-06-01  2019-07-01
12290  2019-06-01  2019-06-29
12314  2019-06-01  2019-07-01
12384  2019-06-01  2019-07-01
12756  2019-06-01  2019-07-01
1364   2019-06-01  2019-07-01
13888  2019-06-01  2019-07-01
1556   2019-06-01  2019-07-02
16241  2019-06-01  2019-07-01
1646   2019-06-01  2019-07-01
1872   2019-06-01  2019-07-01
19504  2019-06-01  2019-07-01
2108   2019-06-01  2019-07-01
2155   2019-06-01  2019-07-01
2176   2019-06-01  2019-07-01
2280   2019-06-17  2019-07-01
22850  2019-06-01  2019-07-01
2290   2019-06-01  2019-07-01
24768  2019-06-01  2019-06-14
26297  2019-06-01  2019-06-01
2672   2019-06-01  2019-07-01
26965  2019-06-01  2019-07-01
2775   2019-06-01  2019-07-01
28016  2019-06-02  2019-07-01
2829   2019-06-01  2019-07-01
2998   2019-06-01  2019-07-02
31144  2019-06-01  2019-07-01
3286   2019-06-01  2019-07-01
3579   2019-06-01  2019-07-01
...           ...         ...
5065   2019-06-01 

In [ ]:
pv_system_id = 2089
with pd.HDFStore(OUTPUT_TIMESERIES_FILENAME, mode='r') as store:
    timeseries = store[str(pv_system_id)]

timeseries['inst_power_watt'].plot();

In [17]:
timeseries['inst_power_watt'].isnull().sum()

0

In [14]:
pv_systems_filtered.loc[pv_system_id]

Array Tilt                                      30.0°
Inverter                             Sunny Boy 4000TL
Location                          United Kingdom IP30
Orientation                         South East 135.0°
Panels                     16x235W Sanyo HIT-N235SE10
Shade                                             Low
system_capacity                               3.760kW
system_name                Suffolk-Drinkstone-Woodend
Outputs                            2912 days 00:00:00
Generation                                  27.105MWh
Average                                      9.308kWh
Efficiency                                2.518kWh/kW
system_capacity_kw                               3.76
efficiency_kWh_per_kW                           2.518
system_1_tilt_degrees                              30
system_2_tilt_degrees                             NaN
status_interval_minutes                            10
install_date                      2011-07-13 00:00:00
latitude                    

In [30]:
# Sort indicies and set all types to float64
# TODO: remove duplicate indicies
with pd.HDFStore(OUTPUT_TIMESERIES_FILENAME, mode='a') as store:
    keys = store.keys()
    for key in keys:
        print('\r', key, end='', flush=True)
        if key == '/missing_dates':
            continue
        data = store.select(key=key).sort_index()
        for col in data.columns:
            data[col] = data[col].astype(np.float64)
        store.put(key, data, format='table', append=False)

 /1646

/home/jack/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '11542'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/jack/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '12314'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/jack/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '12756'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though


 /22850

/home/jack/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '1646'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/jack/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '1872'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/jack/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '19504'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  

 /28016

...

/home/jack/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '28016'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/jack/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '2998'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/jack/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '4771'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  

 /8883

/home/jack/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '55005'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/jack/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '5701'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/jack/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '7225'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  

 /missing_dates

/home/jack/anaconda3/lib/python3.7/site-packages/tables/path.py:157: NaturalNameWarning: object name is not a valid Python identifier: '8883'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
